In [31]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

Function to check if all ID in the new data correspond to song and artist in file "id_information_mmrs.tsv" from first assignment

In [32]:
df1 = pd.read_csv('ws23_exercise1/id_information_mmsr.tsv', delimiter='\t')
df2 = pd.read_csv('task2/id_mfcc_stats_mmsr.tsv', delimiter='\t')
id_df1 = df1.iloc[:, 0]
id_df2 = df2.iloc[:, 0]
are_ids_present = id_df1.isin(id_df2)
missing_ids = id_df1[~are_ids_present]

print("All IDs from file 1 are in file 2:", are_ids_present.all())
print("Missing IDs:", missing_ids.tolist())



All IDs from file 1 are in file 2: False
Missing IDs: ['03Oc9WeMEmyLLQbj']


Since in the new dataset we only have IDs we build a helper functipn to retrieve it from song and artist

In [33]:
def find_song_id(artist, song, info_df):
    matching_songs = info_df[(info_df['artist'] == artist) & (info_df['song'] == song)]
    if matching_songs.empty:
        return None
    return matching_songs.iloc[0]['id']

def calculate_cosine_similarity(vector1, vector2):
    return cosine_similarity([vector1], [vector2])[0][0] #---> maybe we'll find a better way, but this works for now

In [66]:
def get_song_artist_by_id(song_id, info_df):
    """
    Retrieve song and artist from ID-INFORMATION DataFrame given the song ID.
    
    :param song_id: The ID of the song.
    :param info_df: The DataFrame containing song IDs along with corresponding artist and song name.
    :return: A tuple (artist, song) if found, otherwise (None, None).
    """
    song_info = info_df[info_df['id'] == song_id]
    
    if not song_info.empty:
        artist = song_info.iloc[0]['artist']
        song = song_info.iloc[0]['song']
        return artist, song
    else:
        return None, None



In [67]:
# Example usage:
# Assuming you have loaded your ID-INFORMATION DataFrame into a variable named info_df
song_id = "uocjUouyUgCu6vzP"
artist, song = get_song_artist_by_id(song_id, info_df)

if artist is not None and song is not None:
    print(f"Artist: {artist}, Song: {song}")
else:
    print("Song ID not found in the database.")


Artist: The Veronicas, Song: 4ever


In [83]:
#load data
info_df = pd.read_csv('ws23_exercise1/id_information_mmsr.tsv', sep='\t')
mfcc_stats_df = pd.read_csv('task2/id_mfcc_stats_mmsr.tsv', sep='\t')
mfcc_bow_df = pd.read_csv('task2/id_mfcc_bow_mmsr.tsv', sep='\t')
blf_df = pd.read_csv('task2/id_blf_logfluc_mmsr.tsv', sep='\t')
ivec_df = pd.read_csv('task2/id_ivec256_mmsr.tsv', sep='\t')
dnn_df = pd.read_csv('task2/id_musicnn_mmsr.tsv', sep='\t')
genre_df = pd.read_csv('task2/id_genres_mmsr.tsv', sep='\t')


In [77]:
#set query song for exaples
query_song = {'id': '3uNmJmetcUAMc9WR', 'artist': ' The Beatles', 'song': 'Strawberry Fields Forever - Remastered 2009', 'album': 'Magical Mystery Tour (Remastered)'}
N = 10  

<details>
<summary style="font-size: larger; color: white; background-color: darkorange; border: 1px solid darkorange; padding: 5px 15px; border-radius: 8px; cursor: pointer;">Task 1: MFCC</summary>

<div style="background-color: rgba(255, 255, 0, 0.2); padding: 10px; border-radius: 5px;">
  
  

</div>
</details>

In [93]:
#TO-DO: make it similar to one from task 1 so doesn't seem we did it with gpt, also less chaotic lol
def find_similar_songs(artist, song, df, top_n=10):
    try:
        query_id = find_song_id(artist, song, info_df) #this retrieve id of the song from artist and title
        query_features = df[df['id'] == query_id].iloc[0, 1:].values
    except IndexError:
        print('Song ID not found!')
        return
    
    
    similarities = df.apply(lambda row: calculate_cosine_similarity(query_features, row[1:].values), axis=1)
    # create a DataFrame with similarities
    result_df = pd.DataFrame({ 'id': df['id'], 'similarity': similarities})
    # exclude the query song and sort by similarity
    result_df = result_df[result_df['id'] != query_id].sort_values(by='similarity', ascending=False)
    #this retrieve the title and artist from the id of the N retrieved songs
    result_df['artist'], result_df['song'] = zip(*result_df['id'].apply(lambda x: get_song_artist_by_id(x, info_df)))
    column_order = ['id', 'artist', 'song', 'similarity'] #rearrange like assignment 1
    result_df = result_df[column_order]
    return result_df.head(top_n)



In [80]:
# Example usage 1st retrieval system
similar_songs = find_similar_songs(artist, song, mfcc_stats_df, top_n=10)
print(f"MSCC {N} recommendations for: {query_song['song']}")
display(similar_songs.style.hide(axis="index"))

MSCC 10 recommendations for: Strawberry Fields Forever - Remastered 2009


id,artist,song,similarity
rqwTHLni7ulf6SqR,Lindsay Lohan,Symptoms of You,0.969893
2gEq8ocry6T8fyFv,Katy Perry,Waking Up in Vegas,0.968996
xf9WfXxEpkwo13Sa,Billy Talent,Burn the Evidence,0.968618
4Vn9VEIE8GRYVEo0,Stereophonics,Local Boy in the Photograph,0.968587
0pKdVXwboxWtasKV,Laura Pausini,Bellissimo così,0.968492
cXObHyfRP9zXpzul,Lisa,Empty MERMAiD,0.968420
rbw6CccHFvjcDTfV,Aerosmith,Lay It Down,0.968012
ZNa38KEncQOJQDPi,BABYMETAL,KARATE,0.967655
aIJRXTjxYPzwX2Y7,Ace of Base,"Always Have, Always Will",0.967221
awAh4Mi8JoKeMJRB,Skillet,Lions,0.967192


In [74]:
# 2nd retr system
similar_songs = find_similar_songs("Laura Pausini", "Tra Te E Il Mare", blf_df, top_n=10)
display(similar_songs.style.hide(axis="index"))

id,artist,song,similarity
F2Y16LT156bwNxJP,the i.l.y's,Itchy Itchy Itchy,0.979979
I6Ds6HkUVLrp7Nfb,Teenage Fanclub,Ain't That Enough,0.979297
rnnVZYpbjo5ya0kc,Daughtry,What I Meant to Say,0.978786
S9nRdLI9yIk2fBLm,Brendan Benson,Tiny Spark,0.977929
kOVK1Bh9ihrB0CkJ,Glee Cast,Somebody to Love,0.977510
2ekVWpFnEgAWatng,Charly Bliss,Heaven,0.977170
xXatmqi10p9emomf,Okkervil River,For Real,0.976445
AJRMp2jBYDQrME9q,Jonas Brothers,Invisible,0.976403
h6UVygOZUZmbO4gX,Wanda,bologna,0.976314
dAkyvtLIefgavKD9,Stream of Passion,The Curse,0.975273


In [75]:
# 3nd retr system
similar_songs = find_similar_songs("Laura Pausini", "Tra Te E Il Mare", ivec_df, top_n=10)
display(similar_songs.style.hide(axis="index"))

id,artist,song,similarity
hyAOwBgnAOL7f66Q,Laura Pausini,Entre Tu Y Mil Mares,0.410236
PqImzEp8jEAUrtx0,Fernanda Takai,Diz que fui por aÍ,0.392705
ac6oJAIFYM730IcS,Mark Ronson,Valerie (feat. Amy Winehouse),0.373606
jCr64GDnzODJJsyx,Fun Boy Three,Our Lips Are Sealed,0.370938
90qHDMeEXOmGjG3W,Laura Pausini,Il mio sbaglio più grande,0.368899
jPQ15ZwL5PrusQZd,The Mamas & the Papas,Dream A Little Dream Of Me,0.366276
8AQ5QPZoIH144sY6,Smokie,Living Next Door to Alice,0.363653
oq6lTb4nLtIrPRHr,Zazie,Je Suis Un Homme,0.350376
5162TsbI8HVFdeiW,Suede,Trash,0.336839
6xHlTE7G5TPIszvP,Emarosa,A Toast To The Future Kids!,0.334675


In [76]:
#4th retr system
similar_songs = find_similar_songs("Laura Pausini", "Tra Te E Il Mare", dnn_df, top_n=10)
display(similar_songs.style.hide(axis="index"))

id,artist,song,similarity
3puEiVevhqrziT7t,Selena,Si Una Vez,0.982757
Qh5XjFCInkeOAYzu,Supergrass,Time,0.981092
5stZqMOVZDKmzPt5,Kacey Musgraves,Slow Burn,0.977675
kTDmcHeyW2hcF6bQ,Michael Jackson,Girl You're So Together,0.975120
305Grk7cARdFuECa,Alicia Keys,Teenage Love Affair,0.972838
4oEK2TmxA9iA2NxY,Rush,Digital Man,0.970699
89HLanWesQTluceT,Sia,Butterflies,0.968502
8mn4GK8r3v50WwQN,Belinda,Sal de Mi Piel,0.968195
pmZr3XGxuiF2FVcY,Iced Earth,Watching Over Me - Remixed & Remastered,0.966656
B3WpZ1qa8SBlbGU2,Dusty Springfield,Son of a Preacher Man,0.966540


____________________________________________________________

In [101]:

def precision_at_k(retrieved_df, query_id, genre_df, k):
    if k == 0:
        return 0
    retrieved_ids = retrieved_df['id'].head(k)
    query_genres = set(genre_df[genre_df['id'] == query_id]['genre'].values[0])

    relevant_retrieved = sum(retrieved_ids.apply(lambda x: bool(set(genre_df[genre_df['id'] == x]['genre'].values[0]).intersection(query_genres))))
    return relevant_retrieved / k

def recall_at_k(retrieved_df, query_id, genre_df, k):
    retrieved_ids = retrieved_df['id'].head(k)
    query_genres = set(genre_df[genre_df['id'] == query_id]['genre'].values[0])
    total_relevant = len(genre_df[genre_df['genre'].apply(lambda x: bool(set(x).intersection(query_genres)))])
    
    if total_relevant == 0:
        return 0

    relevant_retrieved = sum(retrieved_ids.apply(lambda x: bool(set(genre_df[genre_df['id'] == x]['genre'].values[0]).intersection(query_genres))))
    return relevant_retrieved / total_relevant


In [102]:
def evaluate_system(retrieval_function, df, info_df, genre_df, k):
    precisions = []
    recalls = []

    for idx, row in info_df.iterrows():
        artist, song = row['artist'], row['song']
        
        # Retrieve top k similar songs
        similar_songs_df = retrieval_function(artist, song, df, top_n=k)
        
        # Calculate precision and recall
        precisions.append(precision_at_k(similar_songs_df, row['id'], genre_df, k))
        recalls.append(recall_at_k(similar_songs_df, row['id'], genre_df, k))

    avg_precision = sum(precisions) / len(precisions)
    avg_recall = sum(recalls) / len(recalls)

    return avg_precision, avg_recall

# Example usage



In [103]:
avg_precision, avg_recall = evaluate_system(find_similar_songs, mfcc_stats_df, info_df, genre_df, k=10)
print(f"Average Precision@10: {avg_precision}, Average Recall@10: {avg_recall}")

Song ID not found!


TypeError: 'NoneType' object is not subscriptable